### Resolução do desafio técnico BeAnalytics

```
Extração Automatizada de Dados do SteamDB
Este notebook tem como objetivo extrair dados de pico de jogadores do site SteamDB, processá-los e armazená-los em um formato estruturado. Os dados extraídos foram carregados em serviços como Google BigQuery e Google Sheets.

```



##### Instalação e Importação de Bibliotecas


In [56]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from datetime import datetime
import json
import os
from google.cloud import bigquery
from google.oauth2 import service_account
import csv 

#### Urls e cabeçalhos das requisições que serão utilizadas para extrair os dados

In [38]:
URLS = {
    'gameratings': 'https://steamdb.info/stats/gameratings/',
    'globaltopsellers': 'https://steamdb.info/stats/globaltopsellers/'
}

HEADERS = {
    'User-Agent': (
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
        '(KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 OPR/109.0.0.0 (Edition std-1)'
    ),
    'Accept': (
        'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,'
        'image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7'
    ),
    'Accept-Encoding': 'gzip, deflate, br, zstd',
    'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
    'Referer': 'https://steamdb.info/charts/',
    'Sec-Ch-Ua': '"Opera GX";v="109", "Not:A-Brand";v="8", "Chromium";v="123"',
    'Sec-Ch-Ua-Arch': '"x86"',
    'Sec-Ch-Ua-Bitness': '"64"',
    'Sec-Ch-Ua-Full-Version': '"109.0.5097.93"',
    'Sec-Ch-Ua-Full-Version-List': (
        '"Opera GX";v="109.0.5097.93", "Not:A-Brand";v="8.0.0.0", "Chromium";v="123.0.6312.124"'
    ),
    'Sec-Ch-Ua-Mobile': '?0',
    'Sec-Ch-Ua-Model': '""',
    'Sec-Ch-Ua-Platform': '"Windows"',
    'Sec-Ch-Ua-Platform-Version': '"15.0.0"',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1'
}

COOKIES = {
    'SLG_G_WPT_TO': 'pt',
    'SLG_GWPT_Show_Hide_tmp': '1',
    'SLG_wptGlobTipTmp': '1',
    '__Host-cc': 'us',
    'cf_clearance': (
        'Fol6iBezomkbMSDRxzEGcntqaxMi49JrtuocqohAzus-1716068823-1.0.1.1-EwAqAA6pNgJzu8WXg2VzkuD0oMBj3S1JC5ub7Qg8VdS2UEvv'
        'I8nYOIVtV.5KFeSxjMVR80tTwyZ9mtdcZGYoBg'
    ),
    '__cf_bm': (
        'v8Uz9K829LAN3r9ACs35Y.r4xgq6MbiXzftfmptWH6I-1716075030-1.0.1.1-P2PT821NWNcgw7KOaIH88lj6K_1xEwXJbLFIKY4iNPCGKXzm'
        'dQBYu2NWSBpEwC2SfYMEGU6tXD1R4ZlZbH0kDw'
    )
}

In [39]:
COUNTRY_CODE = 'br'
CSV_FILE_PATH = 'dados/combined_price_history.csv'

#### Extração dos dados

In [50]:
def fetch_html(url, headers, cookies):
    """Fetch the HTML content of a webpage."""
    response = requests.get(url, headers=headers, cookies=cookies)
    if response.status_code == 200:
        return response.content
    print(f'Failed to retrieve the webpage. Status code: {response.status_code}')
    return None

def parse_gameratings(html_content):
    """Parse HTML content and extract game ratings."""
    soup = BeautifulSoup(html_content, 'html.parser')
    pattern = re.compile(
        r'<tr class="app" data-appid="(\d+)" data-cache="[^"]+">.*?<td>\s*(\d+)\.\s*</td>.*?<td class="applogo">.*?<a aria-hidden="true" href="([^"]+)">.*?<td>.*?<a href="[^"]+">\s*(.*?)\s*</a>.*?<td class="text-right">\s*([\d,]+)\s*</td>.*?<td class="text-right">\s*([\d,]+)\s*</td>.*?<td class="text-right">\s*([\d,]+)\s*</td>.*?<td class="text-right b">\s*([\d.]+)%\s*</td>',
        re.DOTALL
    )
    matches = pattern.findall(str(soup))
    data = [
        {
            'App ID': match[0],
            'Posição': match[1],
            'Link': 'https://steamdb.info' + match[2],
            'Nome do Jogo': match[3],
            'Avaliações Positivas': match[4].replace(',', ''),
            'Avaliações Negativas': match[5].replace(',', ''),
            'Avaliações Totais': match[6].replace(',', ''),
            'Classificação': match[7]
        }
        for match in matches
    ]
    return pd.DataFrame(data)

def parse_globaltopsellers(html_content):
    """Parse HTML content and extract top sellers data."""
    pattern = re.compile(
        r'<tr class="app" data-appid="(\d+)" data-cache="\d+">\s*<td data-sort="\d+">#(\d+)</td>\s*<td class="applogo text-left">'
        r'<a href="/app/\d+/charts/" aria-hidden="true"><img src="/static/img/applogo.svg" alt=""></a></td>\s*<td>'
        r'<a href="/app/\d+/charts/">(.*?)</a></td>\s*<td data-sort=""></td>\s*<td data-sort="\d+">(.*?)</td>\s*'
        r'<td data-sort="\d+">(.*?)</td>\s*</tr>',
        re.DOTALL
    )
    matches = pattern.findall(html_content.decode('utf-8'))
    data = [
        {
            'App ID': appid,
            'Posição': rank,
            'Nome': name.strip(),
            'Avaliações Positivas': positive.strip().replace(',', ''),
            'Avaliações Negativas': negative.strip().replace(',', '')
        }
        for appid, rank, name, positive, negative in matches
    ]
    return pd.DataFrame(data)

def parse_charts_24h(html_content):
    """Parse HTML content and extract 24-hour peak data."""
    pattern = re.compile(
        r'<tr class="app" data-appid="(\d+)"[^>]*>.*?'
        r'<td class="applogo text-left"><a href="[^"]+"><img[^>]+></a></td>.*?'
        r'<td><a href="[^"]+">(.*?)</a></td>.*?'
        r'<td data-sort="(\d+)">.*?</td>.*?'
        r'<td data-sort="(\d+)">.*?</td>.*?'
        r'<td data-sort="(\d+)">.*?</td>',
        re.DOTALL
    )
    matches = pattern.findall(html_content.decode('utf-8'))
    data = [
        {
            'App ID': match[0],
            'Nome do Jogo': match[1],
            'Atual': match[2],
            'Pico 24hrs': match[3],
            'Pico de todos os tempos': match[4]
        }
        for match in matches
    ]
    return pd.DataFrame(data)

def fetch_price_history(app_id, max_retries=5):
    """Fetch price history for a given app ID."""
    url = f'https://steamdb.info/api/GetPriceHistory/?appid={app_id}&cc={COUNTRY_CODE}'
    HEADERS['Referer'] = f'https://steamdb.info/app/{app_id}/'
    
    for attempt in range(max_retries):
        response = requests.get(url, headers=HEADERS, cookies=COOKIES)
        if response.status_code == 200:
            data = response.json()
            
            # Processar os dados para o formato desejado
            price_history = {
                datetime.utcfromtimestamp(entry['x'] / 1000).strftime('%Y-%m-%d'): entry['y']
                for entry in data['data']['history']
            }

            return price_history
        
        elif response.status_code in {429, 500, 502, 503, 504}:
            time.sleep(2 ** attempt)  # Exponencial backoff
        else:
            return None
    
    print(f"Excedido o número máximo de tentativas para o App ID {app_id}.")
    return None

def save_to_csv(df, file_path):
    """Save the DataFrame to a CSV file."""
    df.to_csv(file_path, index=False)
    print(f'Data saved to {file_path} successfully.')
    
def main():
    # Carregar App IDs do arquivo picos_historicos.csv
    picos = pd.read_csv("dados/picos_historicos.csv")
    app_ids = picos['App ID'][:10]  # Pegar os primeiros 10 app IDs

    # Carregar dados existentes de histórico de preços
    if os.path.exists(CSV_FILE_PATH):
        df_existing = pd.read_csv(CSV_FILE_PATH)
        price_histories = df_existing.to_dict('records')
    else:
        price_histories = []

    # Buscar HTML para cada URL e salvar os dados extraídos
    for key, url in URLS.items():
        html_content = fetch_html(url, HEADERS, COOKIES)
        if html_content:
            if key == 'gameratings':
                df = parse_gameratings(html_content)
                save_to_csv(df, 'dados/extracted_game_ratings_5.csv')
            elif key == 'globaltopsellers':
                df = parse_globaltopsellers(html_content)
                save_to_csv(df, 'dados/globaltopsellers_extracted_5.csv')
            elif key == 'charts_24h':
                df = parse_charts_24h(html_content)
                save_to_csv(df, 'dados/picos_historicos_5.csv')
                print("Dados salvos em 'dados/picos_historicos.csv' com sucesso.")

    # Buscar histórico de preços para cada App ID
    for app_id in app_ids:
        price_history = fetch_price_history(app_id)
        if price_history:
            price_histories.append({
                'App ID': app_id,
                'price_history': json.dumps(price_history)  # Converter o dicionário para string JSON
            })
            save_to_csv(pd.DataFrame(price_histories), CSV_FILE_PATH)  # Salvar após cada iteração
            time.sleep(1)  # Esperar 1 segundo entre as requisições para evitar taxa de limite

    print("All data processed and saved successfully.")

if __name__ == '__main__':
    main()



Data saved to dados/extracted_game_ratings_5.csv successfully.
Data saved to dados/globaltopsellers_extracted_5.csv successfully.
Data saved to dados/picos_historicos_5.csv successfully.
Dados salvos em 'dados/picos_historicos.csv' com sucesso.
Data saved to dados/combined_price_history.csv successfully.
Data saved to dados/combined_price_history.csv successfully.
Data saved to dados/combined_price_history.csv successfully.
Data saved to dados/combined_price_history.csv successfully.
Data saved to dados/combined_price_history.csv successfully.
Data saved to dados/combined_price_history.csv successfully.
Data saved to dados/combined_price_history.csv successfully.
All data processed and saved successfully.


#### Enviando os dados ao Google Big Query

In [57]:
# Função para corrigir problemas comuns no CSV
def clean_csv(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        reader = csv.reader(file)
        rows = list(reader)

    # Verificar e corrigir problemas de aspas
    cleaned_rows = []
    for row in rows:
        cleaned_row = [value.replace('"', '').replace('\n', ' ').strip() for value in row]
        cleaned_rows.append(cleaned_row)

    with open(file_path, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerows(cleaned_rows)

    print(f'CSV cleaned: {file_path}')

# Função para carregar dados de um CSV para o BigQuery
def upload_to_bigquery(csv_file_path, table_id):
    credential_path = "big_query/credentials.json"

# Carregar as credenciais do arquivo JSON
    credentials = service_account.Credentials.from_service_account_file(credential_path)
    client = bigquery.Client(credentials=credentials, project=credentials.project_id)
    # Limpar o CSV antes de carregar
    clean_csv(csv_file_path)

    # Carregar os dados do CSV para um DataFrame
    df = pd.read_csv(csv_file_path, dtype=str)

    # Configurar o job de carregamento
    job_config = bigquery.LoadJobConfig(
        write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
        source_format=bigquery.SourceFormat.CSV,
        autodetect=True,
    )

    # Carregar os dados do DataFrame para o BigQuery
    job = client.load_table_from_dataframe(df, table_id, job_config=job_config)
    job.result()  # Esperar a conclusão do job

    # Verificar se a tabela foi carregada com sucesso
    destination_table = client.get_table(table_id)
    print(f"Carregado {destination_table.num_rows} linhas na tabela {table_id}.")

In [58]:
# Definir o caminho para o arquivo de credenciais JSON
credential_path = "big_query/credentials.json"

# Carregar as credenciais do arquivo JSON
credentials = service_account.Credentials.from_service_account_file(credential_path)

# Criar um cliente do BigQuery
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

# Testar a conexão listando conjuntos de dados no projeto
datasets = list(client.list_datasets())
if datasets:
    print("Conjuntos de dados no projeto:")
    for dataset in datasets:
        print(f" - {dataset.dataset_id}")
else:
    print("Nenhum conjunto de dados encontrado no projeto.")


# Definir os IDs do projeto, conjunto de dados e tabelas
project_id = 'desafio-beanalytics'
dataset_id = 'jogos'

# Tabelas e arquivos CSV correspondentes
tables_and_files = {
    'game_ratings': 'dados/extracted_game_ratings.csv',
    'topsellers': 'dados/globaltopsellers_extracted.csv',
    'picos_historicos': 'dados/picos_historicos.csv',
    'price_histories': 'dados/combined_price_history.csv'
}

# Carregar cada tabela para o BigQuery
for table_name, csv_file_path in tables_and_files.items():
    full_table_id = f'{project_id}.{dataset_id}.{table_name}'
    upload_to_bigquery(csv_file_path, full_table_id)



Conjuntos de dados no projeto:
 - jogos
CSV cleaned: dados/extracted_game_ratings.csv
Carregado 250 linhas na tabela desafio-beanalytics.jogos.game_ratings.
CSV cleaned: dados/globaltopsellers_extracted.csv
Carregado 984 linhas na tabela desafio-beanalytics.jogos.topsellers.
CSV cleaned: dados/picos_historicos.csv
Carregado 7342 linhas na tabela desafio-beanalytics.jogos.picos_historicos.
CSV cleaned: dados/combined_price_history.csv
Carregado 193 linhas na tabela desafio-beanalytics.jogos.price_histories.
